In [1]:
# Data stuff
import numpy as np
import pandas as pd

# Visual stuff
from IPython.display import display
# Configs 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt

# Random state seed
rand_state=42

# ML stuff
import sklearn
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import shap

In [2]:
dataset = pd.read_csv('data_gen/dataset_mean_undersampled.csv')
# Ensure correct types
dataset = dataset.astype(dtype={
    'age':float,
    'gender':int,
    'expired':int,
    'P-glucose':float,
    'blood_pressure_systoliskt':float,
    'blood_pressure_diastoliskt':float,
    'BMI':float
    
})
# Order ints(categorical variables, first) and floats(number variables, last)
dataset = dataset[[
    'gender',
    'I109',
    'E119',
    'E669',
    'I259',
    'I252',
    'I209',
    'E660',
    'E118',
    'I639',
    'E113',
    'expired',
    'age',
    'P-glucose',
    'blood_pressure_systoliskt',
    'blood_pressure_diastoliskt',
    'BMI',
]]

In [3]:
# Generate data set withouth categories(all numbers)
Y_no_cat = dataset.expired.values
X_no_cat = dataset.drop(columns=['expired']).values

# dataset as is, but target variable dropped(hospital expire flag)
dataset_no_target = dataset.drop(columns=['expired'])

# Feture names and categorical feature names
feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() + dataset_no_target.select_dtypes(exclude='int').columns.values.tolist() 
cat_feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() 

# Generate data set with categories(int type required)
dataframe_int_list = dataset_no_target.select_dtypes(include='int').values.tolist()
dataframe_no_int_list = dataset_no_target.select_dtypes(exclude='int').values.tolist()
Y = dataset.expired.values.tolist()
X = []
for i,v in enumerate(dataframe_int_list):
    X = X + [v+dataframe_no_int_list[i]]

# Generate categorical feature indicies
cat_features_indices=list(range(0,len(dataframe_int_list[0])))

In [4]:
# Function that w
def strat_cv_it(classifier, params, uses_cat, param_comb ):
    folds = 3
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = rand_state)
    
    # Perform fit and scoring
    if uses_cat:
        random_search = RandomizedSearchCV( classifier, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X, Y),random_state = rand_state, refit=True, verbose=3, error_score=0.0 )
        random_search.fit(X, Y)     
    else:
        random_search = RandomizedSearchCV( classifier, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X_no_cat, Y_no_cat),random_state = rand_state, refit=True, verbose=3,error_score=0.0)
        random_search.fit(X_no_cat, Y_no_cat)
    
    # Display results and return best model
    display(random_search.best_score_)
    display(random_search.best_params_)
    display(pd.DataFrame(random_search.cv_results_))
    return random_search.best_estimator_

# Scale of negative class to the positive class(#survived/#died)
scale_pos_weight_min = int( (dataset[dataset.expired==0].shape[0] / dataset[dataset.expired==1].shape[0]) )

# XGboost

In [5]:
xgb_classifier = xgb.XGBClassifier(objective = "binary:logistic",random_state=rand_state)
xgb_params = {
        'learning_rate': (0.01, 0.05,0.1),
        'min_child_weight': [3, 5, 10],
        'gamma': [0.5, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': list(range(5,30)),
      #  'scale_pos_weight':  [0.5,0.75,0.9,1, 1.1, 1.25,1.5]
}
model = strat_cv_it(xgb_classifier ,xgb_params,False,50)
explainer = shap.TreeExplainer(model)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   40.4s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  4.0min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:  4.5min finished


0.8749897215734554

{'subsample': 0.6,
 'min_child_weight': 10,
 'max_depth': 10,
 'learning_rate': 0.1,
 'gamma': 5,
 'colsample_bytree': 0.8}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_min_child_weight,param_max_depth,param_learning_rate,param_gamma,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,1.863043,0.374919,0.026024,0.003652,0.8,10,12,0.1,2,1,"{'subsample': 0.8, 'min_child_weight': 10, 'ma...",0.866250,0.873882,0.867945,0.869359,0.003272,29,0.911756,0.902815,0.905590,0.906720,0.003737
1,3.351985,1.272847,0.032990,0.012707,0.8,10,24,0.01,5,1,"{'subsample': 0.8, 'min_child_weight': 10, 'ma...",0.871001,0.872098,0.866387,0.869828,0.002474,27,0.893017,0.885273,0.892567,0.890285,0.003549
2,3.342857,0.665409,0.031083,0.004441,1,5,25,0.01,1.5,0.6,"{'subsample': 1.0, 'min_child_weight': 5, 'max...",0.875387,0.862767,0.864807,0.867654,0.005531,34,0.912058,0.899364,0.911056,0.907493,0.005763
3,2.991750,0.197241,0.031507,0.001661,1,10,28,0.1,5,0.8,"{'subsample': 1.0, 'min_child_weight': 10, 'ma...",0.874893,0.872915,0.866885,0.871564,0.003406,16,0.898253,0.886163,0.896388,0.893601,0.005315
4,5.179532,2.879745,0.022064,0.013412,0.6,10,10,0.1,5,0.8,"{'subsample': 0.6, 'min_child_weight': 10, 'ma...",0.876182,0.875215,0.873572,0.874990,0.001078,1,0.891248,0.886653,0.890642,0.889515,0.002038
5,3.853434,1.593909,0.039872,0.002149,0.8,10,6,0.1,5,0.8,"{'subsample': 0.8, 'min_child_weight': 10, 'ma...",0.874549,0.874592,0.871992,0.873711,0.001215,8,0.900431,0.890838,0.894738,0.895336,0.003939
6,4.529672,1.593452,0.035148,0.006603,0.8,3,24,0.05,1.5,0.8,"{'subsample': 0.8, 'min_child_weight': 3, 'max...",0.852253,0.864035,0.857124,0.857804,0.004834,43,0.948754,0.943621,0.941505,0.944627,0.003044
7,10.787271,7.814469,0.041096,0.016130,1,5,23,0.05,0.5,0.8,"{'subsample': 1.0, 'min_child_weight': 5, 'max...",0.854640,0.859671,0.856713,0.857008,0.002064,45,0.945179,0.942326,0.939409,0.942305,0.002356
8,17.538629,2.994951,0.041616,0.008327,0.8,3,17,0.05,0.5,1,"{'subsample': 0.8, 'min_child_weight': 3, 'max...",0.845287,0.859327,0.855003,0.853206,0.005871,49,0.960672,0.960510,0.954491,0.958558,0.002876
9,12.272884,5.069210,0.029532,0.012265,0.8,3,26,0.01,1.5,1,"{'subsample': 0.8, 'min_child_weight': 3, 'max...",0.863046,0.866207,0.866430,0.865228,0.001545,39,0.923350,0.922153,0.924192,0.923232,0.000837


In [ ]:
shap_values = explainer.shap_values( X_no_cat)
shap.summary_plot(shap_values,X_no_cat ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names, interaction_index='gender', show=False)
        
    # plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')

# LightGBM

In [7]:
LGB_classifier = lgb.LGBMClassifier()
LGB_params = {
             'num_leaves': [1,5,8,10,15,20,35,40], 
             'min_child_samples': [1,5,10,20,50,100,200,300,400,500], 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': [0.2,0,4,0.5, 0.6, 0.8, 1.0],
             'colsample_bytree': [0.6, 0.8, 1.0],
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
           #  'scale_pos_weight':  [0.5,0.75,0.9,1, 1.1, 1.25,1.5]

}
model = strat_cv_it(LGB_classifier ,LGB_params,True,50)
explainer = shap.TreeExplainer(model)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    4.9s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   20.9s
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:   25.4s finished


0.8728382771622037

{'subsample': 1.0,
 'reg_lambda': 5,
 'reg_alpha': 0,
 'num_leaves': 35,
 'min_child_weight': 0.01,
 'min_child_samples': 200,
 'colsample_bytree': 0.8}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_min_child_weight,param_min_child_samples,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.265160,0.210537,0.015085,0.009349,0.6,50,100,5,0.1,20,0.6,"{'subsample': 0.6, 'reg_lambda': 50, 'reg_alph...",0.816262,0.500000,0.500000,0.605421,0.149088,19,0.827653,0.500000,0.500000,0.609218,0.154457
1,0.004215,0.002913,0.000000,0.000000,4,100,7,10,1,400,0.8,"{'subsample': 4, 'reg_lambda': 100, 'reg_alpha...",0.000000,0.000000,0.000000,0.000000,0.000000,38,0.000000,0.000000,0.000000,0.000000,0.000000
2,1.365374,0.399132,0.022337,0.004233,0.5,100,10,20,10,20,0.6,"{'subsample': 0.5, 'reg_lambda': 100, 'reg_alp...",0.877214,0.859284,0.864764,0.867087,0.007502,9,0.885025,0.879913,0.881865,0.882268,0.002106
3,0.900564,0.802665,0.014615,0.009077,0.5,0.1,1,8,0.001,300,0.6,"{'subsample': 0.5, 'reg_lambda': 0.1, 'reg_alp...",0.809856,0.786765,0.825658,0.807426,0.015971,16,0.806250,0.806245,0.804918,0.805804,0.000627
4,1.493541,0.292488,0.001920,0.000027,0.6,0,50,8,1e-05,20,0.6,"{'subsample': 0.6, 'reg_lambda': 0, 'reg_alpha...",0.857607,0.840256,0.855847,0.851237,0.007798,13,0.852330,0.861707,0.852798,0.855612,0.004315
5,0.004391,0.003114,0.000000,0.000000,0,20,10,15,0.001,1,1,"{'subsample': 0, 'reg_lambda': 20, 'reg_alpha'...",0.000000,0.000000,0.000000,0.000000,0.000000,38,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.003554,0.001911,0.000000,0.000000,0.6,5,5,1,1000,20,0.6,"{'subsample': 0.6, 'reg_lambda': 5, 'reg_alpha...",0.000000,0.000000,0.000000,0.000000,0.000000,38,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.036621,0.029017,0.002925,0.001946,0.5,0.1,100,40,1,300,1,"{'subsample': 0.5, 'reg_lambda': 0.1, 'reg_alp...",0.500000,0.500000,0.500000,0.500000,0.000000,24,0.500000,0.500000,0.500000,0.500000,0.000000
8,1.244488,0.012330,0.022729,0.003308,0.8,0.1,0,10,0.001,100,1,"{'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alp...",0.860445,0.866787,0.868183,0.865138,0.003367,12,0.912074,0.898749,0.910857,0.907227,0.006015
9,0.255451,0.122072,0.032469,0.018336,0.2,20,7,10,1000,50,0.6,"{'subsample': 0.2, 'reg_lambda': 20, 'reg_alph...",0.500000,0.500000,0.500000,0.500000,0.000000,24,0.500000,0.500000,0.500000,0.500000,0.000000


In [ ]:
shap_values = explainer.shap_values(dataset.drop(columns=['expired']))
shap.summary_plot(shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, interaction_index='gender', show=False)
        
  #  plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')